# Finden Sie Ihren Reiseplan | Programme ton itinéraire | Programma un itinerario 

In [ ]:
%%capture
! git lfs pull

In [ ]:
import datetime
import ipydatetime
import networkx as nx
from math import radians, cos, sin, asin, sqrt

import ipywidgets as widgets
from IPython.display import display, clear_output

from planning import *
from planning_helpers import *
from interface_helpers import *

In [ ]:
file = open("../figs/train.jpg", "rb")
image = file.read()
widgets.Image(value=image, format='png', width=1200, height=400)

To use this robust route planner, enter your departure and arrival points as well as the desired time of arrival.
Don't forget to specify the number of trips you want to see, and the certainty of the provided trips. Once you're ready to go, hit `Start!`.

*Note: do not hesitate to press the reset button on the map, as it sometimes loads in a smaller size.*

In [ ]:
# In this notebook, we create the infrastrucure needed for the plannification, and then present the planning algorithm itself. 
# To use this notebook, you just need to run this notebook.
# All the data needed is already stored in git LFS.

In [ ]:
# Read the network stored in lfs
G = nx.read_gpickle("../data/graph.pickle")

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    '''
        Returns the distance between two points in Km
    '''
    R = 6372.8
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))

    return R * c

In [ ]:
def distance_to_zurich_hb(lat, lon):
    ''' 
        Compute the distance from Zürich HB to location passed as parameter
    '''
    return haversine(lat, lon, 47.378177, 8.540192)

In [ ]:
# We create several collections to help the access of stops according to their id or to their name. 
# We also construct a collection containing all the valid stops at less than 15 km from Zürich Main Station.

In [ ]:
valid_stops = set()
stops_by_id = {}
stops_by_name = {}

for stop, data in G.nodes(data = True):
    stops_by_id[stop] = data
    stops_by_name[data['name']] = stop
    if distance_to_zurich_hb(data['lat'], data['lon']) <= 15:
        valid_stops.add(data['name']) 
valid_stops = sorted(list(valid_stops))

In [ ]:
def submit_query(G, src_str, dst_str, arrival_time, n_trips, probability, stops_by_name, stops_by_id):
    '''
        Given the input values and the graph corresponding to the infrastructure, finds the trips, 
        computes their summaries, and displays them in a collapsable accordion view (one item 
        per trip)
    '''
    out.clear_output()
    with out:
        src, dst = stops_by_name[src_str], stops_by_name[dst_str]
        trips = compute_n_trips(n_trips, G, src, dst, arrival_time, probability)

        # Returns tuples like (short summary, long summary, n_transfers, walked distance)
        summaries = [trip_summary(G, list(trip.items()), proba, stops_by_id) for trip, proba in trips]

        # Sort by shortest walked distance then by least number of transfers 
        sorted(summaries, key = lambda s: (s[-2], s[-1]))

        children = [create_trip_view(G, long_summary, nodes) for _, long_summary, nodes, _, _ in summaries]
        accordion = widgets.Accordion(children = children, selected_index = None)
        [accordion.set_title(i, summary[0]) for i, summary in enumerate(summaries)]
        display(accordion)

In [ ]:
n_trip_values = [i for i in range(1, 11)]

layout = widgets.Layout(width='auto', height='30px', padding='0px 20px 40px 0', justify_content = 'space-between') #set width and height

# Setup the widgets required for the interface
src_choices = widgets.Combobox(options= valid_stops, placeholder = 'Choose a stop', ensure_option = True, layout=layout)
dst_choices = widgets.Combobox(options= valid_stops, placeholder = 'Choose a stop', ensure_option = True, layout=layout)
time_picker = ipydatetime.TimePicker(min = datetime.time(6, 0, 0), max = datetime.time(21, 0, 0), layout=layout)
n_trip_choices = widgets.Dropdown(options= n_trip_values, value= n_trip_values[0], layout=layout)
p_choice = widgets.BoundedFloatText(value=50, min=0, max=100, step=1, layout=layout)
button = widgets.Button(description="Start !")

items = [
    widgets.Label('From'), src_choices, 
    widgets.Label('To'), dst_choices, 
    widgets.Label('Before'), time_picker,
    widgets.Label('Number of trips'), n_trip_choices, 
    widgets.Label('With certainty (%)'), p_choice, 
    button
]

grid_box = widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(6, 250px)"))

# Display everything
out = widgets.Output()
display(grid_box)

# Bind the button to the function that will launch the search 
button.on_click(lambda x : submit_query(G, src_choices.value, dst_choices.value, 
                                        str(time_picker.value), n_trip_choices.value, 
                                        p_choice.value/100, stops_by_name, stops_by_id))
display(out)